Импортируем все необходимые библиотеки

In [16]:
pip install natasha scikit-learn

   ---------------------------------------- 0.0/34.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/34.4 MB ? eta -:--:--
   ---------------------------------------- 0.3/34.4 MB ? eta -:--:--
    --------------------------------------- 0.8/34.4 MB 2.8 MB/s eta 0:00:12
   -- ------------------------------------- 2.1/34.4 MB 4.2 MB/s eta 0:00:08
   --- ------------------------------------ 2.6/34.4 MB 3.6 MB/s eta 0:00:09
   --- ------------------------------------ 3.1/34.4 MB 4.2 MB/s eta 0:00:08
   ----- ---------------------------------- 4.5/34.4 MB 3.9 MB/s eta 0:00:08
   ------ --------------------------------- 5.5/34.4 MB 4.4 MB/s eta 0:00:07
   -------- ------------------------------- 7.3/34.4 MB 4.9 MB/s eta 0:00:06
   ---------- ----------------------------- 8.7/34.4 MB 5.1 MB/s eta 0:00:06
   ------------ --------------------------- 10.5/34.4 MB 5.4 MB/s eta 0:00:05
   -------------- ------------------------- 12.1/34.4 MB 5.6 MB/s eta 0:00:04
   ---------------

In [54]:
import string
from natasha import Doc, Segmenter, MorphVocab, NewsEmbedding, NewsMorphTagger
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

1. Лемматизация и стемминг текста

Эта функция выполняет лемматизацию и стемминг для русского текста с помощью библиотеки natasha
1) Текст разбивается на предложения и слова
2)  Слова преобразуются в нормальные формы
3)  Загружаются предобученные векторные представления слов
4)  Анализируется морфология
После создается объект и разбивается на токены, добавляется морфологический анализ
Затем прорабатывается цикл поиска лемм и добавления их в список
В стемменге усекаются слова до основы

In [196]:
def lemm_and_stem(text):
    segmenter = Segmenter()
    Vocab = MorphVocab()
    Embeddings = NewsEmbedding()
    Tagger = NewsMorphTagger(Embeddings)
    Stemmer = SnowballStemmer("russian")
    
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(Tagger)
    
    lems = []
    for token in doc.tokens:
        if token.lemma:  
            lemma = token.lemma
            Vocab.lemmatize(token)  
            lems.append(token.lemma)
        else:
            lems.append(token.text.lower()) 

    stems = [Stemmer.stem(t.text) for t in doc.tokens]
    
    return lems, stems

2. Функция для токенизации всех символов из ASCII
    1) Все символы, не являющиеся частью ASCII, будут проигнорированы.
    2) Текст будет разбит на токены.


In [180]:
def tokenize(text):

    ASII = re.sub(r'[^ -~]+', '', text)
    tokens = ASII.split()

    return tokens

3. Функция для векторизации всех символов из ASCII
    1) Векторизация текста с помощью CountVectorizer из sklearn.
    2) Все символы, не являющиеся частью ASCII, будут проигнорированы.

In [183]:
def vectorize(text):
    
    tokens = tokenize(text)

    vector = CountVectorizer()
    v_text = vector.fit_transform([' '.join(tokens)])

    return v_text.toarray()

4. Токенизация и векторизация текста после лемматизации и стемминга
    1) Лемматизация и стемминг текста.
    2) Токенизация ASCII символов.
    3) Векторизация текста с использованием CountVectorizer.

In [219]:
def v_lemm_stem(lemms):
    embeddings = NewsEmbedding()
    final_vector = []
    
    for lemm in lemms:
        if lemm.isalpha():
            try:
                vector = embeddings[lemm]
                if vector is None:  
                    vector = np.zeros(300)
            except:
                vector = np.zeros(300)
            final_vector.append(vector)
        else:
            final_vector.append(np.zeros(300))
    
    return np.array(vectors)

Русскоязычный текст(лемматизированный и стеммированный)

In [221]:
text = "По вечерам над ресторанами Горячий воздух дик и глух, И правит окриками пьяными Весенний и тлетворный дух."

lemm_text, stem_text = lemm_and_stem(text)
print("Лемматизированный текст:", lemm_text, '\n', '\nCтеммированный текст:', stem_text)

vectors = v_lemm_stem(lemm_text)
print("Векторизация после лемматизации:", vectors)

Лемматизированный текст: ['по', 'вечерам', 'над', 'ресторанами', 'горячий', 'воздух', 'дик', 'и', 'глух', ',', 'и', 'правит', 'окриками', 'пьяными', 'весенний', 'и', 'тлетворный', 'дух', '.'] 
 
Cтеммированный текст: ['по', 'вечер', 'над', 'ресторан', 'горяч', 'воздух', 'дик', 'и', 'глух', ',', 'и', 'прав', 'окрик', 'пьян', 'весен', 'и', 'тлетворн', 'дух', '.']
Векторизация после лемматизации: [[-0.4462508  -0.25771007 -0.10361444 ... -0.12318133  0.17685772
   0.01288834]
 [ 0.06137407  0.01232695  0.18338987 ...  0.42134026 -0.26348916
  -0.38667753]
 [-0.33192572 -0.25423947 -0.31180307 ... -0.83404738 -0.0029812
  -0.16885023]
 ...
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.1074577   0.10019927 -0.17940037 ... -0.64201134  0.19201182
   0.08100268]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]]


In [152]:
text1 = 'Sun of the sleepless! Melancholy star! Whose tearful beam glows tremulously far, That show’st the darkness thou canst not dispel, How like art thou to joy remember’d well!'

tokens = tokenize(text1)
print("\nТокены ASCII:", tokens)

vectorized = vectorize(text1)
print("\nВекторизация ASCII:", vectorized)


Токены ASCII: ['Sun', 'of', 'the', 'sleepless!', 'Melancholy', 'star!', 'Whose', 'tearful', 'beam', 'glows', 'tremulously', 'far,', 'That', 'showst', 'the', 'darkness', 'thou', 'canst', 'not', 'dispel,', 'How', 'like', 'art', 'thou', 'to', 'joy', 'rememberd', 'well!']

Векторизация ASCII: [[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 1 1 1 1]]
